In [7]:
from binance_common.configuration import ConfigurationWebSocketStreams
from binance_sdk_derivatives_trading_usds_futures.derivatives_trading_usds_futures import (
    DerivativesTradingUsdsFutures as UsdFutures,
    ConfigurationRestAPI,
    DERIVATIVES_TRADING_USDS_FUTURES_REST_API_PROD_URL as MAINNET_REST,
    DERIVATIVES_TRADING_USDS_FUTURES_WS_STREAMS_PROD_URL as MAINNET_STREAMS,
)
import ssl
import certifi

ssl_context = ssl.create_default_context(cafile=certifi.where())

configuration = ConfigurationRestAPI(
    api_key='cpP4jcnrP6DA7MDQzXbjGjhGtmn3DtOZ7aPKbwpAG7kWaRPHV2oKVqI20BZr9NnL',
    api_secret='uQT9OowyxhtrFMaTJxGTFHDVecQ2qBoqT56635Wt1r5ztFzezZvL1kLn18q9Csdw',
    base_path=MAINNET_REST
)
configuration_ws_streams = ConfigurationWebSocketStreams(
    stream_url=MAINNET_STREAMS,
    https_agent=ssl_context
)

client = UsdFutures(config_rest_api=configuration, config_ws_streams=configuration_ws_streams)

In [ ]:
from binance_sdk_derivatives_trading_usds_futures.rest_api.models import (
    FuturesAccountBalanceV3Response, StartUserDataStreamResponse
)

response = client.rest_api.futures_account_balance_v3()
data: list[FuturesAccountBalanceV3Response] = response.data()

for item in data:
    print(type(item))

print(response.data())


In [ ]:
from binance_common.models import ApiResponse

from binance_sdk_derivatives_trading_usds_futures.rest_api.models import (
    ExchangeInformationResponse,
)

res: ApiResponse[ExchangeInformationResponse] = client.rest_api.exchange_information()
data = res.data()

for sym in data.symbols:
    print(sym)


In [ ]:


klines_data = client.rest_api.kline_candlestick_data(
    symbol="BTCUSDT",
    interval="5m",
    limit=None,
).data()[:-1]

print(klines_data)

In [ ]:
%pip install --upgrade pip
%pip install --upgrade certifi

In [ ]:
import asyncio
import os
import logging
import ssl
import certifi

from binance_sdk_derivatives_trading_usds_futures.derivatives_trading_usds_futures import (
    DerivativesTradingUsdsFutures,
    DERIVATIVES_TRADING_USDS_FUTURES_WS_STREAMS_PROD_URL,
    ConfigurationWebSocketStreams,
)

# Configure logging
logging.basicConfig(level=logging.INFO)

# Create SSL context
ssl_context = ssl.create_default_context(cafile=certifi.where())

# Create configuration for the WebSocket Streams
configuration_ws_streams = ConfigurationWebSocketStreams(
    stream_url=os.getenv(
        "STREAM_URL", DERIVATIVES_TRADING_USDS_FUTURES_WS_STREAMS_PROD_URL
    ),
    https_agent=ssl_context  # Changed from https_agent
)

# Initialize DerivativesTradingUsdsFutures client
client = DerivativesTradingUsdsFutures(config_ws_streams=configuration_ws_streams)

def _market_stream_handler(stream):
    print(type(stream))


async def kline_candlestick_streams():
    connection = None
    try:
        connection = await client.websocket_streams.create_connection()
        for symbol in ["btcusdt", "ethusdt", "xrpusdt"]:
            stream = await client.websocket_streams.kline_candlestick_streams(
                symbol=symbol,
                interval="1m",
            )
            stream.on("message", callback=_market_stream_handler)

        await asyncio.sleep(5)
        await stream.unsubscribe()
    except Exception as e:
        logging.error(f"kline_candlestick_streams() error: {e}")
    finally:
        if connection:
            await connection.close_connection(close_session=True)


await kline_candlestick_streams()

INFO:root:Connecting to wss://fstream.binance.com/stream with proxy None


INFO:root:Establishing Websocket connection with id 6cf3a88a-dd77-4b29-9b67-16878452afbf to: wss://fstream.binance.com/stream
INFO:root:Subscribing to streams: ['xrpusdt@kline_1m']
INFO:root:Sending message to WebSocket 6cf3a88a-dd77-4b29-9b67-16878452afbf: {'method': 'SUBSCRIBE', 'params': ['xrpusdt@kline_1m'], 'id': 'aed5645a-0323-42c0-ada6-866cb9b83b07'}


<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.KlineCandlestickStreamsResponse'>
<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.KlineCandlestickStreamsResponse'>
<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.KlineCandlestickStreamsResponse'>
<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.KlineCandlestickStreamsResponse'>
<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.KlineCandlestickStreamsResponse'>
<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.KlineCandlestickStreamsResponse'>
<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.Kline

INFO:root:Unsubscribed from stream: xrpusdt@kline_1m


<class 'binance_sdk_derivatives_trading_usds_futures.websocket_streams.models.kline_candlestick_streams_response.KlineCandlestickStreamsResponse'>


INFO:root:WebSocket 6cf3a88a-dd77-4b29-9b67-16878452afbf closed.


In [8]:
def _user_stream_handler(stream) -> None:
    print(type(stream))
    print(stream)


async def user_data_streams():
    await client.websocket_streams.create_connection()
    user_data_response: StartUserDataStreamResponse = client.rest_api.start_user_data_stream().data()
    listen_key = user_data_response.listen_key
    stream = await client.websocket_streams.user_data(listen_key)
    stream.on(event="message", callback=_user_stream_handler)


await user_data_streams()

INFO:root:Connecting to wss://fstream.binance.com/stream with proxy None
INFO:root:Establishing Websocket connection with id d6b12227-39a7-4c39-b1bb-d418dd8710ad to: wss://fstream.binance.com/stream
INFO:root:Subscribing to streams: ['dqE2jpFFrM6rxCo8z7kn4LcSMMKYw4Q39J3Yhc6UbSjCxYCjyzI3iP6u4WlRqNJk']
INFO:root:Sending message to WebSocket d6b12227-39a7-4c39-b1bb-d418dd8710ad: {'method': 'SUBSCRIBE', 'params': ['dqE2jpFFrM6rxCo8z7kn4LcSMMKYw4Q39J3Yhc6UbSjCxYCjyzI3iP6u4WlRqNJk'], 'id': '2a5ffe45-6ab9-4565-97dd-582ada0436b2'}


<class 'dict'>
{'e': 'ACCOUNT_UPDATE', 'T': 1759518194746, 'E': 1759518194746, 'a': {'B': [{'a': 'USDT', 'wb': '30.54568341', 'cw': '30.54568341', 'bc': '-5'}], 'P': [], 'm': 'WITHDRAW'}}
